In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import pickle
# from soynlp.hangle import levenshtein
# # from PreProcessing.find_common_part
# from konlpy.tag import *
# from PreProcessing import find_common_part
import numpy as np
from gensim.models import Word2Vec
from keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# 불러올때    
with open('/content/drive/Shared drives/BigData/team__KU/data/result.txt', 'rb') as f:
    result = pickle.load(f)
    
result_gensim_input = [_.split() for _ in result if _ != '']
result_tokenizer_input = [v for i, v in enumerate(result) if i%2 == 0 and v != '']
ebs_in_result_for_getting_max_len = [v for i, v in enumerate(result_gensim_input) if i%2 == 0 and v != []]
ebs_in_result_gensim_input = [v for i, v in enumerate(result_gensim_input) if i%2 == 0 and v != [] ]
google_in_result_gensim_input = [v for i, v in enumerate(result_gensim_input) if i%2 == 1 and v != []]

# Generate EBS string vectors matrix
ws = 1
es = 30

model_cbow = Word2Vec(
                ebs_in_result_gensim_input, 
                window = ws,
                size =es,
                min_count=1,
                workers = 10
                )

t = Tokenizer()
t.fit_on_texts(result_tokenizer_input)
vocab_size = len(t.word_index) + 1

embedding_matrix = np.zeros((vocab_size, es))
for w, i in t.word_index.items():
    embedding_matrix[i] = model_cbow[w]
print('embedding_matrix.shape :{}'.format(embedding_matrix.shape))

#시퀀스 만들기
sequences = list()
for line in result_tokenizer_input: # 1,214 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[i-9 if i > 9 else 0:i+1]
        sequences.append(sequence)

y = []
for _ in np.array(sequences):
    y.append(_.pop(-1))
    
X = np.array(sequences)
y = np.array(y)
        
max_len=max(len(l) for l in X)
print('X : {}'.format(X.shape))

X = pad_sequences(X, maxlen=max_len, padding='post')

y = to_categorical(y, num_classes=vocab_size)

embedding_layer = Embedding(vocab_size,
                            es,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=False)

model = Sequential()
model.add(embedding_layer)
# y데이터를 분리하였으므로 이제 X데이터의 길이는 기존 데이터의 길이 - 1
model.add(LSTM(128, activation = 'relu'))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, batch_size = 40, epochs=500, verbose=2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


embedding_matrix.shape :(12168, 30)
X : (49851,)
Epoch 1/500
1247/1247 - 28s - loss: 7.7793 - accuracy: 0.0156
Epoch 2/500
1247/1247 - 28s - loss: 7.4347 - accuracy: 0.0197
Epoch 3/500
1247/1247 - 28s - loss: 7.3034 - accuracy: 0.0211
Epoch 4/500
1247/1247 - 28s - loss: 7.1986 - accuracy: 0.0221
Epoch 5/500
1247/1247 - 28s - loss: 7.1040 - accuracy: 0.0225
Epoch 6/500
1247/1247 - 28s - loss: 7.0127 - accuracy: 0.0232
Epoch 7/500
1247/1247 - 28s - loss: 6.9360 - accuracy: 0.0230
Epoch 8/500
1247/1247 - 28s - loss: 6.8588 - accuracy: 0.0233
Epoch 9/500
1247/1247 - 28s - loss: 6.7600 - accuracy: 0.0256
Epoch 10/500
1247/1247 - 29s - loss: 6.6279 - accuracy: 0.0270
Epoch 11/500
1247/1247 - 29s - loss: 6.4628 - accuracy: 0.0287
Epoch 12/500
1247/1247 - 29s - loss: 6.2813 - accuracy: 0.0299
Epoch 13/500
1247/1247 - 29s - loss: 6.0872 - accuracy: 0.0307
Epoch 14/500
1247/1247 - 28s - loss: 5.8629 - accuracy: 0.0340
Epoch 15/500
1247/1247 - 28s - loss: 5.6252 - accuracy: 0.0464
Epoch 16/500
12

In [5]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [11]:
print(sentence_generation(model, t, '다시', 10))
# 다시 한번 첫 시작 지점 뭐라고 이렇게 둘 더한것이십육이 : EBS
# 다시 한번 첫 시작 기점 뭐라고 이렇게 둘 도아것이십육일이 : GOOGLE

다시 그리면 십칠 몇개 됐나요 백팔십 둘레만 했어요 알수 로 이백


# post, e = 500, verbose = 2, 임베딩매트릭스 코드 수정 후, trainable = False

In [7]:
print(sentence_generation(model, t, '주황색', 10))
# 주황색 변해요 길어지죠 빗변 길이 어떻게 되죠 길어집니다
# 주황색 변해요 빗변 길이 너어떻게 되죠 길어집니다

주황색 그리면 됐나요 그리면 십칠 다시 분모 차카 오 구할건데 줄여나가볼거야이각


In [8]:
print(sentence_generation(model, t, '약속', 10))
# 약속 여러분 첫 삼각비 는걸 공부 하기에앞서서 약속 여러분 익히셔야 됩니다
# 약속 여러분 첫 삼각비 란걸 공부 기안써서이 약속 여러분 이틀쉬어야 됩니다

약속 그리면 됐나요 그리면 십칠 다시 좀했는데요 디 레벨 그렇죠그럼탄젠트 바로대입합시다


In [9]:
print(sentence_generation(model, t, '직각삼각형', 10))
# 직각삼각형 만약 우리 이각 기준 각놔 보겠습니다
# 직각삼각형 만약 우리 이각 기준 가기나 보겠습니다

직각삼각형 그리면 평행 원칙 사가 위 뭐라고 된다는 하는데 반복 날수


In [10]:
print(sentence_generation(model, t, '두번째', 10))
# 두번째 탄젠트 에이 했습니다
# 두번째 탄젠트 얘기 했습니다

두번째 그리면 십칠 그리면 됐나요 겨냥 돼길어져이 이용 절대값 삼이 삼각형
